<a href="https://colab.research.google.com/github/2003UJAN/SIH_RE-DACT/blob/main/ReDact_Text%2CImage%2CAudio%2CVideo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
pip install spacy opencv-python moviepy speechrecognition gtts cryptography

In [72]:
pip install pydub ffmpeg

In [73]:
import torch
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline
from pydub import AudioSegment
from pydub.playback import play
import speech_recognition as sr
import cv2
import os
import logging
import moviepy.editor as mp
import speech_recognition as sr
from gtts import gTTS
from cryptography.fernet import Fernet

In [74]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = BertForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [75]:
encryption_key = Fernet.generate_key()
cipher_suite = Fernet(encryption_key)

In [76]:
logging.basicConfig(filename='redaction_tool.log', level=logging.INFO, format='%(asctime)s %(message)s')

In [77]:
def perform_ner(text):
    entities = nlp(text)
    entity_list = [(ent['word'], ent['entity']) for ent in entities]
    return entity_list

In [78]:
def sanitize_input(input_str):
    return input_str.replace(';', '').replace('&', '').replace('|', '').strip()

In [79]:
def redact_text(text, entities, degree):
    for entity, label in entities:
        if degree == 1 and label in ["B-PER", "B-ORG"]:
            text = text.replace(entity, "[REDACTED]")
        elif degree == 2 and label in ["B-PER", "B-ORG", "B-DATE"]:
            text = text.replace(entity, "[REDACTED]")
        elif degree == 3:
            text = text.replace(entity, "[REDACTED]")
    return text

In [80]:
def save_text_file(content, filename="redacted_text.txt"):
    sanitized_filename = sanitize_input(filename)
    with open(sanitized_filename, 'w') as file:
        file.write(content)
    logging.info(f"Redacted text saved as {sanitized_filename}")

In [81]:
def encrypt_file_content(content):
    return cipher_suite.encrypt(content.encode('utf-8'))

In [82]:
def decrypt_file_content(content):
    return cipher_suite.decrypt(content).decode('utf-8')

In [83]:
def redact_image_with_haar(image_path, degree=1):
    sanitized_image_path = sanitize_input(image_path)
    if not os.path.exists(sanitized_image_path):
        logging.error(f"File not found: {sanitized_image_path}")
        return None

    image = cv2.imread(sanitized_image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        if degree == 1:
            face = image[y:y+h, x:x+w]
            blurred_face = cv2.GaussianBlur(face, (99, 99), 30)
            image[y:y+h, x:x+w] = blurred_face
        elif degree == 2:
            face = image[y:y+h, x:x+w]
            small_face = cv2.resize(face, (w // 10, h // 10), interpolation=cv2.INTER_LINEAR)
            resized_face = cv2.resize(small_face, (w, h), interpolation=cv2.INTER_NEAREST)
            image[y:y+h, x:x+w] = resized_face
        elif degree == 3:
            image[y:y+h, x:x+w] = 0

    return image

In [84]:
def save_image_file(image, filename="redacted_image.jpg"):
    sanitized_filename = sanitize_input(filename)
    cv2.imwrite(sanitized_filename, image)
    logging.info(f"Redacted image saved as {sanitized_filename}")

In [85]:
def redact_video_with_haar(video_path, degree=1):
    sanitized_video_path = sanitize_input(video_path)
    if not os.path.exists(sanitized_video_path):
        logging.error(f"File not found: {sanitized_video_path}")
        return None

    cap = cv2.VideoCapture(sanitized_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(f'redacted_{os.path.basename(sanitized_video_path)}', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)

        for (x, y, w, h) in faces:
            if degree == 1:
                face = frame[y:y+h, x:x+w]
                blurred_face = cv2.GaussianBlur(face, (99, 99), 30)
                frame[y:y+h, x:x+w] = blurred_face
            elif degree == 2:
                face = frame[y:y+h, x:x+w]
                small_face = cv2.resize(face, (w // 10, h // 10), interpolation=cv2.INTER_LINEAR)
                resized_face = cv2.resize(small_face, (w, h), interpolation=cv2.INTER_NEAREST)
                frame[y:y+h, x:x+w] = resized_face
            elif degree == 3:
                frame[y:y+h, x:x+w] = 0

        out.write(frame)

    cap.release()
    out.release()
    logging.info(f"Redacted video saved as redacted_{os.path.basename(sanitized_video_path)}")

In [86]:
def convert_audio_to_wav(audio_path):
    sanitized_audio_path = sanitize_input(audio_path)
    audio = AudioSegment.from_file(sanitized_audio_path)
    wav_path = f"{os.path.splitext(sanitized_audio_path)[0]}.wav"
    audio.export(wav_path, format="wav")
    return wav_path

def synthesize_speech(text, filename="output.wav"):
    from pydub.generators import Sine

    speech = Sine(440).to_audio_segment(duration=len(text) * 50)  # Fake audio for illustration
    # In reality, you'd replace the Sine generator with real TTS or use another method.
    speech.export(filename, format="wav")

def redact_audio(audio_path, degree=1):
    try:
        wav_audio_path = convert_audio_to_wav(audio_path)
        recognizer = sr.Recognizer()
        audio_file = sr.AudioFile(wav_audio_path)

        with audio_file as source:
            audio_data = recognizer.record(source)

        text = recognizer.recognize_google(audio_data)
        entities = perform_ner(text)
        redacted_text = redact_text(text, entities, degree)

        redacted_audio_path = f'redacted_{os.path.basename(wav_audio_path)}'
        synthesize_speech(redacted_text, redacted_audio_path)

        logging.info(f"Redacted audio saved as {redacted_audio_path}")
    except sr.UnknownValueError:
        logging.error("Could not understand audio")
    except sr.RequestError as e:
        logging.error(f"Error with the speech recognition service; {e}")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

In [87]:
def chatbot():
    print("Welcome to the Redaction Tool Chatbot! How can I help you today?")
    while True:
        print("\nOptions:")
        print("1. Redact Text")
        print("2. Redact Image")
        print("3. Redact Video")
        print("4. Redact Audio")
        print("5. Exit")

        choice = input("Choose an option: ")

        if choice == '1':
            text = input("Please enter the text you would like to redact: ")
            print("Redaction levels:")
            print("1. Redact names of people and organizations.")
            print("2. Redact names of people, organizations, and dates.")
            print("3. Redact all detected entities.")
            degree = int(input("Choose the redaction level (1-3): "))
            entities = perform_ner(text)
            redacted_text = redact_text(text, entities, degree)
            encrypted_text = encrypt_file_content(redacted_text)
            save_text_file(encrypted_text.decode('utf-8'))
            print("Text has been redacted and saved.")

        elif choice == '2':
            image_path = input("Please enter the path of the image you would like to redact: ")
            print("Redaction levels:")
            print("1. Blur detected faces.")
            print("2. Pixelate detected faces.")
            print("3. Black out detected faces.")
            degree = int(input("Choose the redaction level (1-3): "))
            redacted_image = redact_image_with_haar(image_path, degree)
            if redacted_image is not None:
                save_image_file(redacted_image)
                print("Image has been redacted and saved.")

        elif choice == '3':
            video_path = input("Please enter the path of the video you would like to redact: ")
            print("Redaction levels:")
            print("1. Blur detected faces.")
            print("2. Pixelate detected faces.")
            print("3. Black out detected faces.")
            degree = int(input("Choose the redaction level (1-3): "))
            redact_video_with_haar(video_path, degree)
            print("Video has been redacted and saved.")

        elif choice == '4':
            audio_path = input("Please enter the path of the audio file you would like to redact: ")
            print("Redaction levels:")
            print("1. Redact names of people and organizations in the audio.")
            print("2. Redact names of people, organizations, and dates in the audio.")
            print("3. Redact all detected entities in the audio.")
            degree = int(input("Choose the redaction level (1-3): "))
            redact_audio(audio_path, degree)
            print("Audio has been redacted and saved.")

        elif choice == '5':
            print("Goodbye!")
            break

        else:
            print("Invalid choice. Please choose a valid option.")

In [ ]:
if __name__ == "__main__":
    chatbot()

Welcome to the Redaction Tool Chatbot! How can I help you today?

Options:
1. Redact Text
2. Redact Image
3. Redact Video
4. Redact Audio
5. Exit
Choose an option: 4
Please enter the path of the audio file you would like to redact: /content/Ujan_Voice_3.mp4
Redaction levels:
1. Redact names of people and organizations in the audio.
2. Redact names of people, organizations, and dates in the audio.
3. Redact all detected entities in the audio.
Choose the redaction level (1-3): 1
Audio has been redacted and saved.

Options:
1. Redact Text
2. Redact Image
3. Redact Video
4. Redact Audio
5. Exit
